In [ ]:
import influxdb_client
import pandas as pd

In [ ]:
TOKEN = '80pzM9UJharBRVfSbBIfecydGJsX80ihx2IZsd7UmAZUhUumZTIiX9jUqQNytXPxeVuZlgLmHAY6JOyHnLZuJw=='
URL = 'http://localhost:8086'
ORG = 'VCT'
BUCKET = 'HOUSE_HOLD_ENERGY'

In [ ]:
client = influxdb_client.InfluxDBClient(url=URL, token=TOKEN, org=ORG,  timeout=30000)

In [ ]:
query_api = client.query_api()

In [ ]:
query = f'''
from(bucket: "{BUCKET}")
  |> range(start: 2007-01-01T00:00:00Z, stop: 2007-12-31T00:00:00Z)
  |> filter(fn: (r) => r["_measurement"] == "house_energy")
  |> filter(fn: (r) => r["_field"] == "Global_active_power" or 
                      r["_field"] == "Global_reactive_power" or 
                      r["_field"] == "Voltage" or 
                      r["_field"] == "Global_intensity" or 
                      r["_field"] == "Sub_metering_1" or 
                      r["_field"] == "Sub_metering_2" or 
                      r["_field"] == "Sub_metering_3")
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''

In [ ]:
house_hold_energy_2007 = query_api.query_data_frame(query=query)

In [ ]:
house_hold_energy_2007.info()

#### Limpando dataframe

##### Retirando dados desnecessários

In [ ]:
house_hold_energy_2007.drop(columns=['result', 'table', '_start', '_stop', '_measurement'], inplace=True)
house_hold_energy_2007.head(10)

In [ ]:
columns_float = list(house_hold_energy_2007.iloc[:, 1:].columns)
columns_float

#### Encontrando dados nulos dados nulos

In [ ]:
rows_null = house_hold_energy_2007.query('Global_active_power == "?"').index
rows_null

In [ ]:
house_hold_energy_2007.iloc[rows_null].head(10)

In [ ]:
house_hold_energy_2007.replace("?", 0.0, inplace=True)
house_hold_energy_2007.iloc[rows_null].head(10)

In [ ]:
house_hold_energy_2007.fillna(0.0, inplace=True)
house_hold_energy_2007.iloc[rows_null].head(10)

In [ ]:
house_hold_energy_2007.interpolate(inplace=True)
house_hold_energy_2007.iloc[rows_null]

#### Convertendo colunas numericas

In [ ]:
house_hold_energy_2007[columns_float] = house_hold_energy_2007[columns_float].astype(float)

In [ ]:
house_hold_energy_2007.info()

#### Adicionando coluna de consumo total (Global_active_power * 1000 / 60 - Sub_metering_1 - Sub_metering_2 - Sub_metering_3)

In [ ]:
house_hold_energy_2007["Energy_consumed"] = house_hold_energy_2007["Global_active_power"] * 1000 / 60 - house_hold_energy_2007["Sub_metering_1"] - house_hold_energy_2007["Sub_metering_2"] - house_hold_energy_2007["Sub_metering_3"]
house_hold_energy_2007.head(10)

#### Renomeando Sub_meterings

In [ ]:
rename_columns = {
    "Sub_metering_1" : "Kitchen",
    "Sub_metering_2" : "Laundry_Room",
    "Sub_metering_3" : "Heater_conditioner"
}

In [ ]:
house_hold_energy_2007.rename(columns=rename_columns, inplace=True)

In [ ]:
house_hold_energy_2007.head(10)

#### Analisando dias da semana de Janeiro


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
jan_energy = house_hold_energy_2007[house_hold_energy_2007["_time"].dt.month == 1]
jan_energy

In [ ]:
def plot_lines_energy_consume(month_df, day_of_week, day):
    day_week_df = month_df[month_df["_time"].dt.dayofweek == day_of_week][["_time", "Energy_consumed"]]
    day_start = pd.to_datetime(day).date()
    for i in range(4):
        day_stop = (day_start + pd.DateOffset(days=6)).date()
        if day_start.month == month_df['_time'].dt.month[0]:
            week_df = day_week_df[(day_week_df["_time"].dt.date >= day_start) & (day_week_df["_time"].dt.date <= day_stop)]
            week_df.plot(kind="line", x="_time", y="Energy_consumed", figsize=(20,5))
            day_start = day_stop
    plt.show()

In [ ]:
def plot_box_energy_consume(month_df, day_of_week, day):
    day_week_df = month_df[month_df["_time"].dt.dayofweek == day_of_week][["_time", "Energy_consumed"]]
    day_start = pd.to_datetime(day).date()
    list_dfs = []
    for i in range(4):
        day_stop = (day_start + pd.DateOffset(days=6)).date()
        if day_start.month == month_df['_time'].dt.month[0]:
            week_df = day_week_df[(day_week_df["_time"].dt.date >= day_start) & (day_week_df["_time"].dt.date <= day_stop)]
            week_df.plot(kind="box", x="_time", y="Energy_consumed", figsize=(20,5))
            day_start = day_stop
    plt.show()

- Domingos

In [ ]:
plot_lines_energy_consume(jan_energy, 0, '2007-01-01')

#### Interpretando um boxplot
- Posição – Em relação à posição dos dados, observa-se a linha central do retângulo (a mediana ou segundo quartil).
- Dispersão – A dispersão dos dados pode ser representada pelo intervalo interquartílico que é a diferença entre o terceiro quartil e o primeiro quartil (tamanho da caixa), ou ainda pela amplitude que é calculada da seguinte maneira: valor máximo – valor mínimo. Embora a amplitude seja de fácil entendimento, o intervalo interquartílico é uma  estatística mais robusta para medir variabilidade uma vez que não sofre influência de outliers.
- Simetria – Um conjunto de dados que tem uma distribuição simétrica, terá a linha da mediana no centro do retângulo. Quando a linha da mediana está próxima ao primeiro quartil, os dados são assimétricos positivos e quando a posição da linha da mediana é próxima ao terceiro quartil, os dados são assimétricos negativos. Vale ressaltar que a mediana é a medida de tendência central mais indicada quando os dados possuem distribuição assimétrica, uma vez que a média aritmética é influenciada pelos valores extremos.
- Caudas – As linhas que vão do retângulo até aos outliers podem fornecer o comprimento das caudas da distribuição.
- Outliers – Já os outliers indicam possíveis valores discrepantes. No boxplot, as observações são consideradas outliers quando estão abaixo ou acima do limite de detecção de outliers.

##### Verificando outliers, mediana, simetria e dispersao com boxplot

In [ ]:
plot_box_energy_consume(jan_energy, 0, '2007-01-01')

- Segundas

In [ ]:
plot_lines_energy_consume(jan_energy, 1, '2007-01-02')

##### Verificando outliers, mediana, simetria e dispersao com boxplot

In [ ]:
plot_box_energy_consume(jan_energy, 1, '2007-01-02')

- Tercas

In [ ]:
plot_lines_energy_consume(jan_energy, 2, '2007-01-03')

##### Verificando outliers, mediana, simetria e dispersao com boxplot

In [ ]:
plot_box_energy_consume(jan_energy, 2, '2007-01-03')

- Quartas

In [ ]:
plot_lines_energy_consume(jan_energy, 3, '2007-01-04')

##### Verificando outliers, mediana, simetria e dispersao com boxplot

In [ ]:
plot_box_energy_consume(jan_energy, 3, '2007-01-04')

- Quintas

In [ ]:
plot_lines_energy_consume(jan_energy, 4, '2007-01-05')

##### Verificando outliers, mediana, simetria e dispersao com boxplot

In [ ]:
plot_box_energy_consume(jan_energy, 4, '2007-01-05')

- Sextas

In [ ]:
plot_lines_energy_consume(jan_energy, 5, '2007-01-06')

##### Verificando outliers, mediana, simetria e dispersao com boxplot

In [ ]:
plot_box_energy_consume(jan_energy, 5, '2007-01-06')

- Sabados

In [ ]:
plot_lines_energy_consume(jan_energy, 6, '2007-01-07')

##### Verificando outliers, mediana, simetria e dispersao com boxplot

In [ ]:
plot_box_energy_consume(jan_energy, 6, '2007-01-07')

#### Analisando consumo ao decorrer da semana

##### Agrupando dados em dias

In [ ]:
energy_per_day = house_hold_energy_2007.set_index('_time').resample('D').sum()
energy_per_day

In [ ]:
energy_per_day.index = energy_per_day.index.normalize()
energy_per_day.head()

In [ ]:
energy_per_day.index = pd.DatetimeIndex(energy_per_day.index.strftime('%Y-%m-%d'))
energy_per_day

In [ ]:
energy_per_day.reset_index(inplace=True)
energy_per_day

#### Selecionando somente os dados de Janeiro

In [ ]:
jan_energy_per_day = energy_per_day[energy_per_day['_time'].dt.month == 1]
jan_energy_per_day

##### Analisando primeira semana de Janeiro

In [ ]:
range_date = pd.date_range("2007-01-01", "2007-01-07", freq="D")

In [ ]:
jan_first_week = jan_energy_per_day.loc[jan_energy_per_day['_time'].isin(range_date)]

In [ ]:
jan_first_week["_time"]

In [ ]:
jan_first_week.plot(kind='line', x='_time', y='Energy_consumed', figsize=(20, 6));

In [ ]:
range_date = pd.date_range("2007-01-08", "2007-01-15", freq="D")

In [ ]:
jan_second_week = jan_energy_per_day.loc[jan_energy_per_day['_time'].isin(range_date)]

In [ ]:
jan_second_week.plot(kind='line', x='_time', y='Energy_consumed', figsize=(20, 6));